In [3]:
import mariadb
import pandas as pd

In [4]:
# Function to establish a connection to MariaDB
def create_db_connection(host, user, password, database):
    try:
        connection = mariadb.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=3306,
        )
        return connection
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [5]:
# Function to execute a SQL query and return results as a Pandas DataFrame
def query_to_dataframe(connection, query):
    try:
        if connection is not None:
            cursor = connection.cursor()
            cursor.execute(query)
            result = cursor.fetchall()
            columns = [i[0] for i in cursor.description]
            df = pd.DataFrame(result, columns=columns)
            return df
        else:
            print("No database connection available.")
            return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [6]:
# Function to insert a Pandas DataFrame into a MariaDB table
def dataframe_to_table(connection, dataframe, table_name):
    try:
        if connection is not None:
            cursor = connection.cursor()

            # Create the table using the DataFrame's structure
            create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
            columns = [f"{col} VARCHAR(255)" for col in dataframe.columns]
            create_table_query += ", ".join(columns) + ")"
            cursor.execute(create_table_query)
            
            # Insert data into the table
            for _, row in dataframe.iterrows():
                values = tuple(row)
                placeholders = ",".join(["%s"] * len(values))
                insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
                cursor.execute(insert_query, values)
            connection.commit()
            print(f"Data inserted into {table_name} successfully.")
        else:
            print("No database connection available.")
    except Exception as e:
        print(f"Error: {str(e)}")

In [7]:
# Replace these with your MariaDB connection details
host = "10.54.209.120"
user = "ICDBSEL"
password = "Santro@123"
database = "DM"

# Create a connection
connection = create_db_connection(host, user, password, database)

In [9]:
# Close the connection when done
if connection is not None:
    connection.close()